In [4]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from datetime import datetime,date,timedelta
import pickle

In [5]:
station = 'KCASANTA106'
staname = 'brommer'
now = datetime.now().date()
sdate = datetime(now.year-2,now.month-5,now.day-5).date()
datediff = (now-sdate).days

In [6]:
allwind = pd.read_pickle(f'{staname}_raw_{sdate}_{now-timedelta(days=1)}.pkl')

In [7]:
allwind[::len(allwind)//3]

,Date,Time,Season,Month,Daypart,Temp,Wind,Azimuth,Speed,Gust
0,2020-05-21,12:04 AM,Spring,May,Overnight,53.8,East,90.0,0.0,0.0
262,2021-03-06,9:54 PM,Spring,Mar,Evening,47.9,NW,315.0,0.0,0.0
217,2022-01-06,6:07 PM,Winter,Jan,Evening,53.1,ENE,67.5,2.0,6.0


In [8]:
def dftostrFF(df):
    print(df.reset_index(drop=False).to_string(index=False))
def dftostrTT(df):
    print(df.reset_index(drop=True).to_string(index=True))
def dftostrTF(df):
    print(df.reset_index(drop=True).to_string(index=False))
def dftostrFT(df):
    print(df.reset_index(drop=False).to_string(index=True))

In [9]:
ordir={"North":0.0,
    "NNE":22.5,
    "NE": 45.0,
    "ENE":67.5,
    "East":90.0,
    "ESE":112.5,
    "SE":135.0,
    "SSE":157.5,
    "South":180,
    "SSW":202.5,
    "SW":225.0,
    "WSW":247.5,
    "West":270.0,
    "WNW":292.5,
    "NW":315.0,
    "NNW":337.5}
revdir = [(value, key) for key, value in ordir.items()]
print('mapping of 16 angular wind directions to 16 compass points:\n')
print(pd.DataFrame(revdir,columns=['Azimuth','Compass']).to_string(index=False))
revord = dict(revdir)

mapping of 16 angular wind directions to 16 compass points:

 Azimuth Compass
     0.0   North
    22.5     NNE
    45.0      NE
    67.5     ENE
    90.0    East
   112.5     ESE
   135.0      SE
   157.5     SSE
   180.0   South
   202.5     SSW
   225.0      SW
   247.5     WSW
   270.0    West
   292.5     WNW
   315.0      NW
   337.5     NNW


In [10]:
def ordinalify(df,groupcol,grouporder):
    avgdir = df.groupby(groupcol)['Azimuth'].mean()[grouporder]
    avgint = avgdir.astype(int)
    clsdir = round(avgdir/22.5,0)*22.5
    dford = pd.concat([pd.concat([pd.DataFrame(avgint),pd.DataFrame(clsdir)],axis=1),clsdir.map(revord)],axis=1)
    dford.columns=['Azimuth','Closest','Compass']
    dford = dford.reset_index(drop=False)
    return dford

In [11]:
def groupavg(df,grpcols,sortcols=None,ordcols=None):
    dfavg = df.groupby(grpcols)[['Azimuth','Speed']].mean().round(1)
    dfavg['Compass'] = (round(dfavg['Azimuth']/22.5,0)*22.5).map(revord) 
    dfavg.reset_index(drop=False,inplace=True)
    if ordcols != None:
        for i in range(len(grpcols)):
            dfavg[grpcols[i]] = pd.Categorical(dfavg[grpcols[i]], ordcols[i])
    if sortcols != None:
        return dfavg.sort_values(sortcols).reset_index(drop=True)
    return dfwavg

In [16]:
groupavg(nnwind,['Season','Daypart'],['Season','Daypart'],[orderszn,orderday])

,Season,Daypart,Azimuth,Speed,Compass
0,Winter,Overnight,171.5,1.2,South
1,Winter,Morning,166.0,1.3,SSE
2,Winter,Midday,174.8,3.8,South
3,Winter,Afternoon,193.9,2.8,SSW
4,Winter,Evening,175.6,1.5,South
5,Spring,Overnight,135.6,1.1,SE
6,Spring,Morning,135.9,1.5,SE
7,Spring,Midday,160.2,4.4,SSE
8,Spring,Afternoon,153.3,4.9,SSE
9,Spring,Evening,153.9,2.8,SSE


In [16]:
print("average wind speed by direction:\n")
dfs = allwind.groupby('Wind')['Speed'].mean().round(1)
dfs = dfs.reset_index(drop=False)
dfs['Wind'] = pd.Categorical(dfs['Wind'], list(ordir.keys()))
dftostrTF(dfs.sort_values('Wind'))

print("ascending avg wind speed by direction:\n")
dftostrTF(dfs.sort_values('Speed'))

average wind speed by direction:

 Wind  Speed
North    1.8
  NNE    1.5
   NE    1.2
  ENE    1.8
 East    1.9
  ESE    2.4
   SE    1.8
  SSE    1.6
South    2.3
  SSW    3.1
   SW    2.7
  WSW    2.0
 West    1.4
  WNW    2.4
   NW    2.5
  NNW    2.0
  NaN    1.7
ascending avg wind speed by direction:

 Wind  Speed
   NE    1.2
 West    1.4
  NNE    1.5
  SSE    1.6
  NaN    1.7
   SE    1.8
  ENE    1.8
North    1.8
 East    1.9
  NNW    2.0
  WSW    2.0
South    2.3
  ESE    2.4
  WNW    2.4
   NW    2.5
   SW    2.7
  SSW    3.1


In [19]:
print("difference in monthly average azimuth without vs with wind speed as weights:\n")
dfdiff=pd.concat([pd.Series(ordermth),(ordinalify(allwind,'Month',ordermth)).Azimuth.subtract((groupwavg(nnwind,'Month',ordermth)).Azimuth)],axis=1)
dfdiff.columns=['Month','Azimuth']
dftostrTF(dfdiff)
print("\n\ndifference in seasonal average azimuth without vs with wind speed as weights:\n")
dfdiff=pd.concat([pd.Series(orderszn),(ordinalify(allwind,'Season',orderszn)).Azimuth.subtract((groupwavg(nnwind,'Season',orderszn)).Azimuth)],axis=1)
dfdiff.columns=['Season','Azimuth']
dftostrTF(dfdiff)

difference in monthly average azimuth without vs with wind speed as weights:

Month  Azimuth
  Jan       32
  Feb       27
  Mar       -1
  Apr      -97
  May      -17
  Jun        4
  Jul      -15
  Aug      -32
  Sep      -29
  Oct      -27
  Nov       -8
  Dec       18


difference in seasonal average azimuth without vs with wind speed as weights:

Season  Azimuth
Winter       11
Spring      -21
Summer       -6
  Fall      -32


In [129]:
print('Speed-weighted Azimuth(Wazm),grouped by Daypart,sorted ascending by Azimuth:\n')
multigrpwavg(nnwind,['Daypart'],'Wazm')

Speed-weighted Azimuth(Wazm),grouped by Daypart,sorted ascending by Azimuth:



,Daypart,Wazm,Compass
0,Overnight,145,SE
1,Morning,150,SSE
2,Evening,162,SSE
3,Afternoon,178,South
4,Midday,180,South


In [130]:
print('Speed-weighted Azimuth(Wazm),grouped by Daypart,sorted explicitly by Daypart:\n')
multigrpwavg(nnwind,['Daypart'],'Daypart',[orderday])

Speed-weighted Azimuth(Wazm),grouped by Daypart,sorted explicitly by Daypart:



,Daypart,Wazm,Compass
0,Overnight,145,SE
1,Morning,150,SSE
2,Midday,180,South
3,Afternoon,178,South
4,Evening,162,SSE


In [125]:
print('Speed-weighted Azimuth(Wazm),grouped by Month,sorted ascending by Azimuth:\n')
multigrpwavg(nnwind,['Month'],'Wazm')

Speed-weighted Azimuth(Wazm),grouped by Month,sorted ascending by Azimuth:



,Month,Wazm,Compass
0,Apr,132,SE
1,Sep,155,SSE
2,Aug,163,SSE
3,Jul,163,SSE
4,Oct,168,SSE
5,Dec,172,South
6,Jun,173,South
7,Jan,174,South
8,May,177,South
9,Nov,180,South


In [123]:
print('Speed-weighted Azimuth(Wazm),grouped by Month,sorted ascending by Month:\n')
multigrpwavg(nnwind,['Month'],'Month',[ordermth])

Speed-weighted Azimuth(Wazm),grouped by Month,sorted ascending by Month:



,Month,Wazm,Compass
0,Jan,174,South
1,Feb,204,SSW
2,Mar,189,South
3,Apr,132,SE
4,May,177,South
5,Jun,173,South
6,Jul,163,SSE
7,Aug,163,SSE
8,Sep,155,SSE
9,Oct,168,SSE


In [128]:
print('Speed-weighted Azimuth(Wazm),grouped by Season,sorted explicitly by Season:\n')
multigrpwavg(nnwind,['Season'],'Season',[orderszn])

Speed-weighted Azimuth(Wazm),grouped by Season,sorted explicitly by Season:



,Season,Wazm,Compass
0,Winter,185,South
1,Spring,166,SSE
2,Summer,166,SSE
3,Fall,164,SSE


In [126]:
print('Speed-weighted Azimuth(Wazm),grouped by Season,sorted ascending by Azimuth:\n')
multigrpwavg(nnwind,['Season'],'Wazm')

Speed-weighted Azimuth(Wazm),grouped by Season,sorted ascending by Azimuth:



,Season,Wazm,Compass
0,Fall,164,SSE
1,Spring,166,SSE
2,Summer,166,SSE
3,Winter,185,South


In [117]:
print('Speed-weighted Azimuth(Wazm),grouped by Season and Daypart,sorted ascending by Wazm:\n')
multigrpwavg(nnwind,['Season','Daypart'],'Wazm')

Speed-weighted Azimuth(Wazm),grouped by Season and Daypart,sorted ascending by Wazm:



,Season,Daypart,Wazm,Compass
0,Fall,Overnight,126,SE
1,Spring,Morning,139,SE
2,Spring,Overnight,141,SE
3,Fall,Evening,144,SE
4,Summer,Overnight,147,SSE
5,Summer,Morning,147,SSE
6,Fall,Morning,149,SSE
7,Summer,Evening,149,SSE
8,Winter,Overnight,162,SSE
9,Winter,Morning,170,South


In [116]:
print('Speed-weighted Azimuth(Wazm),grouped by Season and Daypart, sorted by their explicit order:\n')
multigrpwavg(nnwind,['Season','Daypart'],['Season','Daypart'],[orderszn,orderday])

Speed-weighted Azimuth(Wazm),grouped by Season and Daypart, sorted by their explicit order:



,Season,Daypart,Wazm,Compass
0,Winter,Overnight,162,SSE
1,Winter,Morning,170,South
2,Winter,Midday,184,South
3,Winter,Afternoon,208,SSW
4,Winter,Evening,193,SSW
5,Spring,Overnight,141,SE
6,Spring,Morning,139,SE
7,Spring,Midday,172,South
8,Spring,Afternoon,172,South
9,Spring,Evening,179,South


In [115]:
def multigrpwavg(df,grpcols,sortcols=None,ordcols=None):
    df['Wazm'] = df['Speed']*df['Azimuth']
    dfgsum = df.groupby(grpcols).sum()
    dfgsum['Wazm'] = (dfgsum['Wazm']/dfgsum['Speed']).astype(int)
    dfwavg = dfgsum[['Wazm']]
    dfwavg['Compass'] = (round(dfwavg['Wazm']/22.5,0)*22.5).map(revord) 
    dfwavg.reset_index(drop=False,inplace=True)
    if ordcols != None:
        for i in range(len(grpcols)):
            dfwavg[grpcols[i]] = pd.Categorical(dfwavg[grpcols[i]], ordcols[i])
    if sortcols != None:
        return dfwavg.sort_values(sortcols).reset_index(drop=True)
    return dfwavg

In [13]:
nnwind = allwind[allwind['Azimuth'].notnull()]

In [17]:
def groupwavg(df,groupcol,grouporder):
    wgtdir = (df['Speed']).multiply(df['Azimuth'])
    dfwgt = pd.concat([nnwind,pd.DataFrame(wgtdir,columns=['Wgtdir'])],axis=1)[[groupcol,'Speed','Wgtdir']]
    dfsum = dfwgt.groupby(groupcol).sum()
    wavgd = (dfsum.Wgtdir/dfsum.Speed)
    wintd = wavgd.astype(int)
    wclsd = round(wavgd/22.5,0)*22.5
    wcomp = wclsd.map(revord)
    dfwavg = pd.concat([pd.concat([wintd,wclsd],axis=1),wcomp],axis=1)
    dfwavg.columns = ['Azimuth','Closest','Compass']
    dfwavg = dfwavg.reset_index(drop=False)
    dfwavg[groupcol] = pd.Categorical(dfwavg[groupcol], grouporder)
    return dfwavg.sort_values(groupcol)

In [11]:
print("unweighted average wind azimuth direction by season:\n")
dftostrTF(ordinalify(allwind,'Season',orderszn))
print("\n\nunweighted average wind azimuth direction by month:\n")
dftostrTF(ordinalify(allwind,'Month',ordermth))

unweighted average wind azimuth direction by season:

Season  Azimuth  Closest Compass
Winter      175    180.0   South
Spring      145    135.0      SE
Summer      160    157.5     SSE
  Fall      153    157.5     SSE


unweighted average wind azimuth direction by month:

Month  Azimuth  Closest Compass
  Jan      164    157.5     SSE
  Feb      190    180.0   South
  Mar      171    180.0   South
  Apr      107    112.5     ESE
  May      157    157.5     SSE
  Jun      167    157.5     SSE
  Jul      158    157.5     SSE
  Aug      157    157.5     SSE
  Sep      148    157.5     SSE
  Oct      153    157.5     SSE
  Nov      160    157.5     SSE
  Dec      173    180.0   South


In [15]:
orderszn = ['Winter','Spring','Summer','Fall']
orderday = ['Overnight','Morning','Midday','Afternoon','Evening']
ordermth = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

In [23]:
print("ascending avg wind speed by season and daypart:\n")
dftostrTF(dfs.sort_values('Speed'))
print("\nascending avg wind azimuth by season and daypart:\n")
dftostrTF(dfs.sort_values('Azimuth'))

ascending avg wind speed by season and daypart:

Season   Daypart  Azimuth  Speed
  Fall Overnight    140.3    0.7
Summer Overnight    157.1    0.8
  Fall   Morning    141.6    1.0
Spring Overnight    135.6    1.1
Summer   Morning    148.3    1.2
Winter   Morning    166.0    1.3
Winter Overnight    171.5    1.3
Spring   Morning    135.9    1.5
Winter   Evening    175.6    1.5
  Fall   Evening    148.2    1.6
Summer   Evening    154.4    2.6
Winter Afternoon    193.9    2.8
Spring   Evening    153.9    2.8
  Fall    Midday    173.8    3.3
  Fall Afternoon    174.1    3.3
Summer    Midday    179.4    3.8
Winter    Midday    174.8    3.8
Spring    Midday    160.2    4.4
Summer Afternoon    168.0    4.6
Spring Afternoon    153.3    4.9

ascending avg wind azimuth by season and daypart:

Season   Daypart  Azimuth  Speed
Spring Overnight    135.6    1.1
Spring   Morning    135.9    1.5
  Fall Overnight    140.3    0.7
  Fall   Morning    141.6    1.0
  Fall   Evening    148.2    1.6
Summer  

In [137]:
print("average wind Azimuth and Speed by season and daypart:\n")
dfs = allwind.groupby(['Season','Daypart'])[['Azimuth','Speed']].mean().round(1)
dfs = dfs.reset_index(drop=False)
dfs['Season'] = pd.Categorical(dfs['Season'], ['Winter','Spring','Summer','Fall'])
dfs['Daypart'] = pd.Categorical(dfs['Daypart'], ['Overnight','Morning','Midday','Afternoon','Evening'])
dfs['Compass'] = (round(dfs['Azimuth']/22.5,0)*22.5).map(revord)
dfs = dfs.sort_values(['Season','Daypart']).reset_index(drop=True)
dfs

average wind Azimuth and Speed by season and daypart:



,Season,Daypart,Azimuth,Speed,Compass
0,Winter,Overnight,171.5,1.3,South
1,Winter,Morning,166.0,1.3,SSE
2,Winter,Midday,174.8,3.8,South
3,Winter,Afternoon,193.9,2.8,SSW
4,Winter,Evening,175.6,1.5,South
5,Spring,Overnight,135.6,1.1,SE
6,Spring,Morning,135.9,1.5,SE
7,Spring,Midday,160.2,4.4,SSE
8,Spring,Afternoon,153.3,4.9,SSE
9,Spring,Evening,153.9,2.8,SSE


In [141]:
groupavg(allwind,['Season','Daypart'],[orderszn,orderday],['Season','Daypart'])

TypeError: Parameter 'categories' must be list-like, was 'Season'

In [ ]:
def compgrpwavg(df,groupcols,)

In [18]:
print("speed weighted average wind azimuth direction by Season:\n")
dftostrTF(groupwavg(nnwind,'Season',['Winter','Spring','Summer','Fall']))
print("\n\nspeed weighted average wind azimuth direction by Month:\n")
dftostrTF(groupwavg(nnwind,'Month',ordermth))

speed weighted average wind azimuth direction by Season:

Season  Azimuth  Closest Compass
Winter      185    180.0   South
Spring      166    157.5     SSE
Summer      166    157.5     SSE
  Fall      164    157.5     SSE


speed weighted average wind azimuth direction by Month:

Month  Azimuth  Closest Compass
  Jan      174    180.0   South
  Feb      204    202.5     SSW
  Mar      189    180.0   South
  Apr      132    135.0      SE
  May      177    180.0   South
  Jun      173    180.0   South
  Jul      163    157.5     SSE
  Aug      163    157.5     SSE
  Sep      155    157.5     SSE
  Oct      168    157.5     SSE
  Nov      180    180.0   South
  Dec      172    180.0   South


In [56]:
nnwind2['Wazm'] = nnwind['Speed']*nnwind['Azimuth']
dfgsum = nnwind2.groupby(['Season','Daypart']).sum()
dfgsum['Wazm'] = (dfgsum['Wazm']/dfgsum['Speed']).astype(int)
dfwavg = dfgsum[['Wazm']]
dfwavg.reset_index(drop=False,inplace=True)
dfwavg['Season'] = pd.Categorical(dfwavg['Season'], ['Winter','Spring','Summer','Fall'])
dfwavg['Daypart'] = pd.Categorical(dfwavg['Daypart'], ['Overnight','Morning','Midday','Afternoon','Evening'])
dfwavg.sort_values(['Season','Daypart'])

,Season,Daypart,Wazm
19,Winter,Overnight,162
18,Winter,Morning,170
17,Winter,Midday,184
15,Winter,Afternoon,208
16,Winter,Evening,193
9,Spring,Overnight,141
8,Spring,Morning,139
7,Spring,Midday,172
5,Spring,Afternoon,172
6,Spring,Evening,179


In [58]:
nnwind2['Wazm'] = nnwind['Speed']*nnwind['Azimuth']
dfgsum = nnwind2.groupby('Season').sum()
dfgsum['Wazm'] = (dfgsum['Wazm']/dfgsum['Speed']).astype(int)
dfwavg = dfgsum[['Wazm']]
dfwavg.reset_index(drop=False,inplace=True)
dfwavg['Season'] = pd.Categorical(dfwavg['Season'], ['Winter','Spring','Summer','Fall'])
dfwavg.sort_values('Season')

,Season,Wazm
3,Winter,185
1,Spring,166
2,Summer,166
0,Fall,164


In [31]:
nnwind2[:-100:10]

,Date,Time,Season,Month,Daypart,Temp,Wind,Azimuth,Speed,Gust,Wgtdir
0,2020-05-21,12:04 AM,Spring,May,Overnight,53.8,East,90.0,0.0,0.0,0.00
10,2020-05-21,12:54 AM,Spring,May,Overnight,52.6,East,90.0,0.0,0.0,0.00
20,2020-05-21,1:44 AM,Spring,May,Overnight,51.3,East,90.0,0.0,0.0,0.00
30,2020-05-21,2:34 AM,Spring,May,Overnight,50.5,East,90.0,0.0,0.0,0.00
40,2020-05-21,3:24 AM,Spring,May,Overnight,50.0,East,90.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...
144,2022-10-25,12:04 PM,Fall,Oct,Midday,56.4,SSW,202.5,3.9,8.0,789.75
154,2022-10-25,12:54 PM,Fall,Oct,Midday,56.8,ESE,112.5,4.3,8.9,483.75
164,2022-10-25,1:44 PM,Fall,Oct,Midday,57.9,ESE,112.5,3.3,7.0,371.25
174,2022-10-25,2:34 PM,Fall,Oct,Afternoon,59.3,ESE,112.5,2.3,8.0,258.75


In [28]:
nnwind2=nnwind

In [9]:
print("average wind speed by season and daypart:\n")
dfs = allwind.groupby(['Season','Daypart'])['Speed'].mean().round(1)
dfs = dfs.reset_index(drop=False)
dfs['Season'] = pd.Categorical(dfs['Season'], ['Winter','Spring','Summer','Fall'])
dfs['Daypart'] = pd.Categorical(dfs['Daypart'], ['Overnight','Morning','Midday','Afternoon','Evening'])
dftostrTF(dfs.sort_values(['Season','Daypart']))

average wind speed by season and daypart:

Season   Daypart  Speed
Winter Overnight    1.3
Winter   Morning    1.3
Winter    Midday    3.8
Winter Afternoon    2.8
Winter   Evening    1.5
Spring Overnight    1.1
Spring   Morning    1.5
Spring    Midday    4.4
Spring Afternoon    4.9
Spring   Evening    2.8
Summer Overnight    0.8
Summer   Morning    1.2
Summer    Midday    3.8
Summer Afternoon    4.6
Summer   Evening    2.6
  Fall Overnight    0.7
  Fall   Morning    1.0
  Fall    Midday    3.3
  Fall Afternoon    3.3
  Fall   Evening    1.6
